In [3]:
import pandas as pd      
from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline 

In [4]:
df = pd.read_csv("car_pred.csv")

In [5]:
df.head()

,make_model,hp_kW,km,age,Gearing_Type,Gears,Type,price
0,Audi A1,66.0,56013.0,3.0,Automatic,7.0,Used,15770
1,Audi A1,141.0,80000.0,2.0,Automatic,7.0,Used,14500
2,Audi A1,85.0,83450.0,3.0,Automatic,7.0,Used,14640
3,Audi A1,66.0,73000.0,3.0,Automatic,6.0,Used,14500
4,Audi A1,66.0,16200.0,3.0,Automatic,7.0,Used,16790


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
hp_kW,15909.0,88.493746,26.671943,40.0,66.0,85.0,103.0,294.0
km,15909.0,32092.436526,36982.838757,0.0,1913.0,20410.0,46900.0,317000.0
age,15909.0,1.389591,1.121355,0.0,0.0,1.0,2.0,3.0
Gears,15909.0,5.937268,0.704943,5.0,5.0,6.0,6.0,8.0
price,15909.0,18020.241876,7377.200813,4950.0,12850.0,16900.0,21900.0,74600.0


In [5]:
X = df.drop(columns = ["price"])
y = df.price

In [6]:
cat_onehot = ['make_model', 'Type', 'Gearing_Type']


column_trans = make_column_transformer((OneHotEncoder(handle_unknown="ignore", sparse=False), cat_onehot), 
                                       remainder='passthrough') 

In [7]:
operations = [("OneHot_Ordinal_Encoder", column_trans), ("scaler", MinMaxScaler()), ("Lasso", Lasso(alpha=1.02))]
final_model = Pipeline(steps=operations)

In [8]:
final_model.fit(X, y)

Pipeline(steps=[('OneHot_Ordinal_Encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['make_model', 'Type',
                                                   'Gearing_Type'])])),
                ('scaler', MinMaxScaler()), ('Lasso', Lasso(alpha=1.02))])

In [9]:
import pickle
pickle.dump(final_model, open("lasso_model", 'wb'))

In [10]:
loaded_model = pickle.load(open("lasso_model", "rb"))

In [11]:
my_dict = {
    "make_model": 'Audi A3',
    "hp_kW": 66,
    "km": 17000,
    "age": 2,
    "Gearing_Type": "Automatic",
    "Gears": 7,
    "Type":"Used"
}

In [12]:
new_obs = pd.DataFrame([my_dict])
new_obs

,make_model,hp_kW,km,age,Gearing_Type,Gears,Type
0,Audi A3,66,17000,2,Automatic,7,Used


In [13]:
loaded_model.predict(new_obs)

array([18859.56224265])